In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from multiprocessing import Pool
%matplotlib inline
from scipy.optimize import curve_fit 
from scipy.optimize import root
import scipy.integrate as integrate
import scipy.special as special
from scipy.optimize import fsolve
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam
from keras.layers import BatchNormalization
import keras.backend as K
import csv
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report



INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [48]:
periodlength=10
theta=.8
epochnumber=500

#Reading in the output of the R script since
# Python's neural nets' options are superior.
pathhh='C:\\Documents\\HypTests\\csvstoPython\\'

BORSto500test = pd.read_csv (pathhh+'BORSto500test.csv')
BORSto500train = pd.read_csv (pathhh+'BORSto500train.csv')
BORStoBORStest = pd.read_csv (pathhh+'BORStoBORStest.csv')
BORStoBORStrain = pd.read_csv (pathhh+'BORStoBORStrain.csv')
MPDCto500test = pd.read_csv (pathhh+'MPDCto500test.csv')
MPDCto500train = pd.read_csv (pathhh+'MPDCto500train.csv')
MPDCtoBORStest = pd.read_csv (pathhh+'MPDCtoBORStest.csv')
MPDCtoBORStrain = pd.read_csv (pathhh+'MPDCtoBORStrain.csv')


# Deep Learning Techniques

In [56]:
def fit_nn(X_train,X_test,y_train,y_test): 
    global epochnumber
    global theta1
    global periodlength
    
    y_train_cat= to_categorical(y_train)
    y_test_cat= to_categorical(y_test)

    X_train_val = X_train.values # This step is needed to write "X_train_t" later
    X_test_val= X_test.values

    X_train_t = X_train_val.reshape(X_train_val.shape[0], periodlength, 1)
    X_test_t = X_test_val.reshape(X_test_val.shape[0], periodlength, 1)
    early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)
    K.clear_session()

    model = Sequential()
    model.add(Dense(20, input_shape=(periodlength,), activation='tanh'))
    model.add(Dense(10, activation='relu'))
    #model.add(Dense(10, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train_cat, epochs=epochnumber, verbose=0)
    y_pred_cat = model.predict(X_test)

    y_pred_class=[]


    for i in range(len(y_pred_cat)):
        if y_pred_cat[i][1]>theta1:
            y_pred_class.append(1)
        else:
            y_pred_class.append(0)
    y_pred_class= np.array(y_pred_class)
    print("Neural Network Results")
    print(confusion_matrix(y_test,y_pred_class))
    print(classification_report(y_test,y_pred_class))

    

In [57]:
# Batch Normalizer

In [58]:
def fit_lstm_bn(X_train,X_test,y_train,y_test):
    global epochnumber
    global theta2
    global periodlength
    
    y_train_cat= to_categorical(y_train)
    y_test_cat= to_categorical(y_test)

    X_train_val = X_train.values # This step is needed to write "X_train_t" later
    X_test_val= X_test.values

    X_train_t = X_train_val.reshape(X_train_val.shape[0], periodlength, 1)
    X_test_t = X_test_val.reshape(X_test_val.shape[0], periodlength, 1)

    K.clear_session()
    model = Sequential()



    model = Sequential()
    model.add(LSTM(10, input_shape=(periodlength, 1), activation="tanh"))
    model.add(BatchNormalization())
    model.add(Dense(20, activation='tanh'))
    #model.add(Dense(10, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train_t, y_train_cat, epochs=epochnumber,
          batch_size=32, verbose=0)
    y_pred_cat = model.predict(X_test_t)
    print(y_pred_cat[20])
    y_pred_class1=[]

    for i in range(len(y_pred_cat)):
        if y_pred_cat[i][1]>theta2:
            y_pred_class1.append(1)
        else:
            y_pred_class1.append(0)
    y_pred_class1= np.array(y_pred_class1)

    print(f"LSTM Method with Batch Normalizer...\n")
    print(confusion_matrix(y_test,y_pred_class1))
    print(classification_report(y_test,y_pred_class1))


In [59]:
def predict_based_on(train,test):
    X_train,X_test,y_train,y_test=train.drop("target",axis=1),test.drop("target",axis=1),train["target"],test["target"]
    
    fit_nn(X_train,X_test,y_train,y_test)
    fit_lstm_bn(X_train,X_test,y_train,y_test)

In [78]:
theta1=.9
theta2=.8
predict_based_on(BORStoBORStrain,BORStoBORStest)

Neural Network Results
[[229  21]
 [ 50   9]]
              precision    recall  f1-score   support

           0       0.82      0.92      0.87       250
           1       0.30      0.15      0.20        59

    accuracy                           0.77       309
   macro avg       0.56      0.53      0.53       309
weighted avg       0.72      0.77      0.74       309

[0.86242086 0.13757914]
LSTM Method with Batch Normalizer...

[[227  23]
 [ 50   9]]
              precision    recall  f1-score   support

           0       0.82      0.91      0.86       250
           1       0.28      0.15      0.20        59

    accuracy                           0.76       309
   macro avg       0.55      0.53      0.53       309
weighted avg       0.72      0.76      0.73       309



In [79]:
theta1=.9
theta2=.8
predict_based_on(MPDCtoBORStrain,MPDCtoBORStest)

Neural Network Results
[[189  66]
 [ 33  26]]
              precision    recall  f1-score   support

           0       0.85      0.74      0.79       255
           1       0.28      0.44      0.34        59

    accuracy                           0.68       314
   macro avg       0.57      0.59      0.57       314
weighted avg       0.74      0.68      0.71       314

[0.859488   0.14051196]
LSTM Method with Batch Normalizer...

[[242  13]
 [ 46  13]]
              precision    recall  f1-score   support

           0       0.84      0.95      0.89       255
           1       0.50      0.22      0.31        59

    accuracy                           0.81       314
   macro avg       0.67      0.58      0.60       314
weighted avg       0.78      0.81      0.78       314



In [89]:
theta1=.9
theta2=.8
predict_based_on(BORSto500train,BORSto500test)

Neural Network Results
[[232  44]
 [ 25   8]]
              precision    recall  f1-score   support

           0       0.90      0.84      0.87       276
           1       0.15      0.24      0.19        33

    accuracy                           0.78       309
   macro avg       0.53      0.54      0.53       309
weighted avg       0.82      0.78      0.80       309

[0.9356345  0.06436551]
LSTM Method with Batch Normalizer...

[[184  92]
 [ 19  14]]
              precision    recall  f1-score   support

           0       0.91      0.67      0.77       276
           1       0.13      0.42      0.20        33

    accuracy                           0.64       309
   macro avg       0.52      0.55      0.48       309
weighted avg       0.82      0.64      0.71       309



In [80]:
theta1=.9
theta2=.8
predict_based_on(MPDCto500train,MPDCto500test)

Neural Network Results
[[199  82]
 [ 18  15]]
              precision    recall  f1-score   support

           0       0.92      0.71      0.80       281
           1       0.15      0.45      0.23        33

    accuracy                           0.68       314
   macro avg       0.54      0.58      0.51       314
weighted avg       0.84      0.68      0.74       314

[0.864248   0.13575204]
LSTM Method with Batch Normalizer...

[[240  41]
 [ 26   7]]
              precision    recall  f1-score   support

           0       0.90      0.85      0.88       281
           1       0.15      0.21      0.17        33

    accuracy                           0.79       314
   macro avg       0.52      0.53      0.53       314
weighted avg       0.82      0.79      0.80       314

